In [1]:
import requests
import hashlib
import time
import datetime
from google.colab import userdata
import pandas as pd
import sqlite3

In [2]:
public_key=userdata.get('public_key_marvel')
private_key=userdata.get('private_key_marvel')
timestamp = str(time.time())
md5_hash= timestamp + private_key + public_key
md5_hash = hashlib.md5(md5_hash.encode("utf-8")).hexdigest()

In [8]:
# Modifique a URL para o endpoint de personagens
url = f"https://gateway.marvel.com/v1/public/characters/1011334/comics?ts={timestamp}&apikey={public_key}&hash={md5_hash}"

# Código de tratamento dos dados
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    print(f"Total de personagens: {data['data']['total']}")
    for char in data['data']['results']:
        print(char)

Total de personagens: 12
{'id': 22506, 'digitalId': 10949, 'title': 'Avengers: The Initiative (2007) #19', 'issueNumber': 19, 'variantDescription': '', 'description': '', 'modified': '2017-08-16T12:11:05+0000', 'isbn': '', 'upc': '5960606084-01911', 'diamondCode': 'SEP082362', 'ean': '', 'issn': '', 'format': 'Comic', 'pageCount': 32, 'textObjects': [{'type': 'issue_solicit_text', 'language': 'en-us', 'text': 'SECRET INVASION Tie-In!\r<br>"V-S DAY"\r<br>It\'s been leading to this since the Hank Pym Skrull first came up with the idea for a Fifty State Initiative.  This is the final assault in the Secret Invasion, a nation-wide plan that will test the limits of 3-D MAN\'S superhuman militia, THE KILL KREW ARMY! Join 3-D MAN, CLOUD 9, KOMODO, HARDBALL, and heroes around America in the battle that will decide the fate of the planet and the future of the Initiative program.  Win or lose, there\'s no turning back.  After today, everything changes.\r<br>Rated T+ ...$2.99\r<br>'}, {'type': 'is

# Criação do banco de dados para os characters

In [10]:
DB_NAME = 'marvel_characters_teste.db'
MAX_RETRIES = 3  # Tentativas por requisição
DELAY = 1  # Atraso entre requisições em segundos

In [11]:
 # Função para pegar todos os personagens
def get_all_characters():
     conn = sqlite3.connect(DB_NAME)
     cursor = conn.cursor()

     offset = 0
     total = None
     inserted = 0

     while True:
         for attempt in range(MAX_RETRIES):
             try:
                 # Autenticação
                 ts = str(time.time())
                 hash_input = ts + private_key + public_key
                 md5_hash = hashlib.md5(hash_input.encode()).hexdigest()

                 params = {
                     'ts': ts,
                     'apikey': public_key,
                     'hash': md5_hash,
                     'limit': 100,
                     'offset': offset
                 }

                 response = requests.get(
                     'https://gateway.marvel.com/v1/public/characters',
                     params=params,
                     timeout=10
                 )
                 response.raise_for_status()

                 data = response.json()
                 total = data['data']['total'] if total is None else total
                 results = data['data']['results']

                 # Inserir no banco
                 for char in results:
                     cursor.execute('''
                         INSERT OR REPLACE INTO characters
                         VALUES (?, ?, ?, ?, ?, ?)
                     ''', (
                         char['id'],
                         char['name'],
                         char.get('description', ''),
                         char['modified'],
                         char['comics']['available'],
                         char['comics']['collectionURI'],
                     ))

                 conn.commit()
                 inserted += len(results)
                 print(f"Progresso: {inserted}/{total} ({inserted/total:.1%})")

                 # Verificar se terminou
                 if offset + len(results) >= total:
                     print("\nTodos os personagens foram armazenados!")
                     return

                 offset += 100
                 time.sleep(DELAY)
                 break  # Sai do loop de tentativas

             except requests.exceptions.RequestException as e:
                 print(f"Erro na requisição (tentativa {attempt+1}): {str(e)}")
                 time.sleep(2 ** attempt)  # Backoff exponencial
                 continue
         else:
             print("Falha após múltiplas tentativas. Abortando...")
             break

     conn.close()

In [12]:
def get_all_comics():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    offset = 0
    total = None
    inserted = 0
    DELAY = 1  # Atraso entre requisições

    while True:
        for attempt in range(MAX_RETRIES):
            try:
                # Autenticação
                ts = str(time.time())
                hash_input = ts + private_key + public_key
                md5_hash = hashlib.md5(hash_input.encode()).hexdigest()

                params = {
                    'ts': ts,
                    'apikey': public_key,
                    'hash': md5_hash,
                    'limit': 100,
                    'offset': offset,
                    'orderBy': 'modified'  # Melhora a paginação
                }

                response = requests.get(
                    'https://gateway.marvel.com/v1/public/comics',
                    params=params,
                    timeout=10
                )
                response.raise_for_status()

                data = response.json()
                total = data['data']['total'] if total is None else total
                results = data['data']['results']

                # Inserir quadrinhos no banco
                for comic in results:
                    # Extrair dados das coleções
                    characters_uri = comic.get('characters', {}).get('collectionURI', '')
                    events_uri = comic.get('events', {}).get('collectionURI', '')
                    creators_uri = comic.get('creators', {}).get('collectionURI', '')
                    stories_uri = comic.get('stories', {}).get('collectionURI', '')

                    cursor.execute('''
                        INSERT OR REPLACE INTO comics
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        comic['id'],
                        comic.get('title', 'Sem título'),
                        comic.get('pageCount', 0),
                        comic.get('modified', ''),
                        characters_uri,
                        events_uri,
                        creators_uri,
                        stories_uri
                    ))

                conn.commit()
                inserted += len(results)
                print(f"Progresso: {inserted}/{total} ({inserted/total:.1%})")

                if offset + len(results) >= total:
                    print("\nTodos os quadrinhos foram armazenados!")
                    return

                offset += 100
                time.sleep(DELAY)
                break

            except requests.exceptions.RequestException as e:
                print(f"Erro na requisição (tentativa {attempt+1}): {str(e)}")
                time.sleep(2 ** attempt)
                continue
        else:
            print("Falha após múltiplas tentativas. Abortando...")
            break

    conn.close()

In [13]:
 # Função para criar o banco
def create_database():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # Tabela principal de personagens
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS characters (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        description TEXT,
        modified TEXT,
        comics_available INTEGER,
        comics_collectionURI TEXT
    )''')

    # Tabela de relacionamento (corrigida)
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS characters_comics (
        id_character INTEGER,
        id_comic INTEGER,
        PRIMARY KEY(id_character, id_comic),
        FOREIGN KEY(id_character) REFERENCES characters(id),
        FOREIGN KEY(id_comic) REFERENCES comics(id)
    )''')

    # Tabela Comics

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS comics (
        id INTEGER PRIMARY KEY,
        title TEXT NOT NULL,
        page_count INTEGER,
        modified TEXT,
        character_collectionURI TEXT,
        events_collectionURI TEXT,
        creator_collectionURI TEXT,
        stories_collectionURI TEXT
    )''')

    conn.commit()
    conn.close()

In [14]:
create_database()
get_all_comics()
get_all_characters()

 # Verificar
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM characters")
print(f"\nTotal no banco: {cursor.fetchone()[0]} personagens")
conn.close()

Progresso: 100/64271 (0.2%)
Progresso: 200/64271 (0.3%)
Progresso: 300/64271 (0.5%)
Progresso: 400/64271 (0.6%)
Erro na requisição (tentativa 1): HTTPSConnectionPool(host='gateway.marvel.com', port=443): Read timed out. (read timeout=10)
Progresso: 500/64271 (0.8%)
Progresso: 600/64271 (0.9%)
Progresso: 700/64271 (1.1%)
Erro na requisição (tentativa 1): HTTPSConnectionPool(host='gateway.marvel.com', port=443): Read timed out. (read timeout=10)
Progresso: 800/64271 (1.2%)
Progresso: 900/64271 (1.4%)
Progresso: 1000/64271 (1.6%)
Progresso: 1100/64271 (1.7%)
Progresso: 1200/64271 (1.9%)
Progresso: 1300/64271 (2.0%)
Progresso: 1400/64271 (2.2%)
Progresso: 1500/64271 (2.3%)
Erro na requisição (tentativa 1): HTTPSConnectionPool(host='gateway.marvel.com', port=443): Read timed out. (read timeout=10)
Progresso: 1600/64271 (2.5%)
Erro na requisição (tentativa 1): HTTPSConnectionPool(host='gateway.marvel.com', port=443): Read timed out. (read timeout=10)
Progresso: 1700/64271 (2.6%)
Progresso: 1

In [ ]:
# import time
# import sqlite3
# import hashlib
# import requests
# from requests.exceptions import ReadTimeout, RequestException

# DB_NAME = 'marvel_characters_teste.db'
# MAX_RETRIES = 3
# BASE_TIMEOUT = 10

# def character_comics():
#     conn = sqlite3.connect(DB_NAME)
#     cursor = conn.cursor()

#     print("[INÍCIO] Conectado ao banco de dados")

#     cursor.execute('SELECT id, comics_collectionURI FROM characters LIMIT 10')
#     resultados = cursor.fetchall()
#     print(f"[BANCO] Total de personagens encontrados: {len(resultados)}")

#     for idx, (id_character, comics_collectionURI) in enumerate(resultados, 1):
#         print(f"\n[PERSONAGEM {idx}/{len(resultados)}] Processando ID: {id_character}")
#         print(f"[URL] Endpoint: {comics_collectionURI}")

#         ts = str(time.time())
#         hash_input = ts + private_key + public_key
#         md5_hash = hashlib.md5(hash_input.encode()).hexdigest()

#         params = {
#             'ts': ts,
#             'apikey': public_key,
#             'hash': md5_hash,
#             'limit': 100,
#         }
#         print(f"[PARÂMETROS] Request configurado com timestamp: {ts}")

#         results = None
#         for attempt in range(MAX_RETRIES):
#             current_timeout = BASE_TIMEOUT * (attempt + 1)
#             print(f"\n[TENTATIVA {attempt + 1}/{MAX_RETRIES}]")
#             print(f"[TIMEOUT] Configurado para: {current_timeout}s")

#             try:
#                 print("[REQUISIÇÃO] Enviando para API...")
#                 response = requests.get(
#                     comics_collectionURI,
#                     params=params,
#                     timeout=current_timeout
#                 )
#                 print(f"[RESPOSTA] Status Code: {response.status_code}")

#                 response.raise_for_status()
#                 data = response.json()
#                 print("[DADOS] Resposta parseada com sucesso")

#                 results = data['data']['results']
#                 print(f"[COMICS] Total encontrado: {len(results)}")
#                 break

#             except ReadTimeout as e:
#                 print(f"[ERRO] Timeout na tentativa {attempt + 1}: {str(e)}")
#                 if attempt == MAX_RETRIES - 1:
#                     print("[FALHA] Máximo de tentativas alcançado")
#                 else:
#                     wait_time = 2 ** attempt
#                     print(f"[AGUARDE] Esperando {wait_time}s para retentativa...")
#                     time.sleep(wait_time)

#             except RequestException as e:
#                 print(f"[ERRO] Falha na requisição: {str(e)}")
#                 continue

#         if results:
#             print(f"[BANCO] Inserindo {len(results)} comics para personagem {id_character}")
#             try:
#                 for char in results:
#                     cursor.execute('''
#                         INSERT OR REPLACE INTO characters_comics
#                         VALUES (?, ?)
#                     ''', (id_character, char['id']))
#                 conn.commit()
#                 print("[BANCO] Dados commitados com sucesso")
#             except sqlite3.Error as e:
#                 print(f"[ERRO BANCO] Falha ao inserir dados: {str(e)}")
#                 conn.rollback()
#         else:
#             print("[AVISO] Nenhum dado para inserir")

#     print("\n[FIM] Processamento concluído para todos os personagens")
#     conn.close()
#     print("[BANCO] Conexão fechada")


In [ ]:
# import time
# import sqlite3
# import hashlib
# import requests
# from requests.exceptions import ReadTimeout, RequestException

# DB_NAME = 'marvel_characters_teste.db'
# MAX_RETRIES = 3
# BASE_TIMEOUT = 10

# def character_comics():
#     conn = sqlite3.connect(DB_NAME)
#     cursor = conn.cursor()

#     print("[INÍCIO] Conectado ao banco de dados")

#     cursor.execute('SELECT id, comics_collectionURI FROM characters LIMIT 50')
#     resultados = cursor.fetchall()
#     print(f"[BANCO] Total de personagens encontrados: {len(resultados)}")

#     for idx, (id_character, comics_collectionURI) in enumerate(resultados, 1):
#         print(f"\n[PERSONAGEM {idx}/{len(resultados)}] Processando ID: {id_character}")
#         print(f"[URL] Endpoint: {comics_collectionURI}")

#         ts = str(time.time())
#         hash_input = ts + private_key + public_key
#         md5_hash = hashlib.md5(hash_input.encode()).hexdigest()

#         params = {
#             'ts': ts,
#             'apikey': public_key,
#             'hash': md5_hash,
#             'limit': 100,
#             'offset': 0  # Novo parâmetro para paginação
#         }

#         all_results = []
#         total_comics = 0

#         while True:
#             results = None
#             for attempt in range(MAX_RETRIES):
#                 current_timeout = BASE_TIMEOUT * (attempt + 1)
#                 print(f"\n[TENTATIVA {attempt + 1}/{MAX_RETRIES}]")
#                 print(f"[TIMEOUT] Configurado para: {current_timeout}s")
#                 print(f"[PAGINAÇÃO] Offset atual: {params['offset']}")

#                 try:
#                     print("[REQUISIÇÃO] Enviando para API...")
#                     response = requests.get(
#                         comics_collectionURI,
#                         params=params,
#                         timeout=current_timeout
#                     )
#                     print(f"[RESPOSTA] Status Code: {response.status_code}")

#                     response.raise_for_status()
#                     data = response.json()

#                     # Captura o total na primeira requisição
#                     if params['offset'] == 0:
#                         total_comics = data['data']['total']
#                         print(f"[COMICS] Total a processar: {total_comics}")

#                     page_results = data['data']['results']
#                     all_results.extend(page_results)
#                     print(f"[PAGINAÇÃO] Comics nesta página: {len(page_results)}")

#                     break  # Sai do loop de tentativas se bem sucedido

#                 except ReadTimeout as e:
#                     print(f"[ERRO] Timeout na tentativa {attempt + 1}: {str(e)}")
#                     if attempt == MAX_RETRIES - 1:
#                         print("[FALHA] Máximo de tentativas alcançado")
#                         break
#                     else:
#                         wait_time = 2 ** attempt
#                         print(f"[AGUARDE] Esperando {wait_time}s para retentativa...")
#                         time.sleep(wait_time)

#                 except RequestException as e:
#                     print(f"[ERRO] Falha na requisição: {str(e)}")
#                     continue

#             # Verifica se há mais páginas
#             if params['offset'] + 100 >= total_comics:
#                 break

#             params['offset'] += 100  # Próxima página
#             time.sleep(0.5)  # Delay entre páginas

#         if all_results:
#             print(f"[BANCO] Inserindo {len(all_results)} comics para personagem {id_character}")
#             try:
#                 # Inserção em lote mais eficiente
#                 dados = [(id_character, comic['id']) for comic in all_results]
#                 cursor.executemany('''
#                     INSERT OR IGNORE INTO characters_comics
#                     VALUES (?, ?)
#                 ''', dados)
#                 conn.commit()
#                 print("[BANCO] Dados commitados com sucesso")
#             except sqlite3.Error as e:
#                 print(f"[ERRO BANCO] Falha ao inserir dados: {str(e)}")
#                 conn.rollback()
#         else:
#             print("[AVISO] Nenhum dado para inserir")

#     print("\n[FIM] Processamento concluído para todos os personagens")
#     conn.close()
#     print("[BANCO] Conexão fechada")

In [ ]:
import time
import sqlite3
import hashlib
import requests
from requests.exceptions import ReadTimeout, RequestException

DB_NAME = 'marvel_characters_teste.db'
MAX_RETRIES = 3
BASE_TIMEOUT = 10

def character_comics():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # Contador de requisições
    request_count = 0
    start_time = time.time()

    print("[INÍCIO] Conectado ao banco de dados")

    cursor.execute('SELECT id, comics_collectionURI FROM characters LIMIT 10')
    resultados = cursor.fetchall()
    print(f"[BANCO] Total de personagens encontrados: {len(resultados)}")

    for idx, (id_character, comics_collectionURI) in enumerate(resultados, 1):
        print(f"\n[PERSONAGEM {idx}/{len(resultados)}] Processando ID: {id_character}")
        print(f"[URL] Endpoint: {comics_collectionURI}")

        ts = str(time.time())
        hash_input = ts + private_key + public_key
        md5_hash = hashlib.md5(hash_input.encode()).hexdigest()

        params = {
            'ts': ts,
            'apikey': public_key,
            'hash': md5_hash,
            'limit': 100,
            'offset': 0
        }

        all_results = []
        total_comics = 0

        while True:
            results = None
            for attempt in range(MAX_RETRIES):
                current_timeout = BASE_TIMEOUT * (attempt + 1)
                print(f"\n[TENTATIVA {attempt + 1}/{MAX_RETRIES}]")
                print(f"[TIMEOUT] Configurado para: {current_timeout}s")
                print(f"[PAGINAÇÃO] Offset atual: {params['offset']}")

                try:
                    print("[REQUISIÇÃO] Enviando para API...")
                    # Incrementa o contador a cada tentativa
                    request_count += 1
                    response = requests.get(
                        comics_collectionURI,
                        params=params,
                        timeout=current_timeout
                    )
                    print(f"[RESPOSTA] Status Code: {response.status_code}")

                    # Monitoramento do rate limit
                    remaining = int(response.headers.get('X-RateLimit-Remaining', 3000))
                    print(f"[RATE LIMIT] Requisições restantes: {remaining}")

                    response.raise_for_status()
                    data = response.json()

                    if params['offset'] == 0:
                        total_comics = data['data']['total']
                        print(f"[COMICS] Total a processar: {total_comics}")

                    page_results = data['data']['results']
                    all_results.extend(page_results)
                    print(f"[PAGINAÇÃO] Comics nesta página: {len(page_results)}")

                    break

                except ReadTimeout as e:
                    print(f"[ERRO] Timeout na tentativa {attempt + 1}: {str(e)}")
                    if attempt == MAX_RETRIES - 1:
                        print("[FALHA] Máximo de tentativas alcançado")
                        break
                    else:
                        wait_time = 2 ** attempt
                        print(f"[AGUARDE] Esperando {wait_time}s para retentativa...")
                        time.sleep(wait_time)

                except RequestException as e:
                    print(f"[ERRO] Falha na requisição: {str(e)}")
                    continue

            # Exibe estatísticas a cada 10 requisições
            if request_count % 10 == 0:
                elapsed = time.time() - start_time
                print(f"\n[ESTATÍSTICAS] Requisições: {request_count} | "
                      f"Tempo decorrido: {elapsed:.2f}s | "
                      f"Req/s: {request_count/elapsed:.2f}")

            if params['offset'] + 100 >= total_comics:
                break

            params['offset'] += 100
            time.sleep(0.5)

        if all_results:
            print(f"[BANCO] Inserindo {len(all_results)} comics para personagem {id_character}")
            try:
                dados = [(id_character, comic['id']) for comic in all_results]
                cursor.executemany('''
                    INSERT OR IGNORE INTO characters_comics
                    VALUES (?, ?)
                ''', dados)
                conn.commit()
            except sqlite3.Error as e:
                print(f"[ERRO BANCO] Falha ao inserir dados: {str(e)}")
                conn.rollback()

    # Relatório final
    total_time = time.time() - start_time
    print(f"\n[RELATÓRIO FINAL]")
    print(f"Total de requisições: {request_count}")
    print(f"Tempo total: {total_time:.2f} segundos")
    print(f"Média de requisições/s: {request_count/total_time:.2f}")

    conn.close()
    print("[BANCO] Conexão fechada")

In [ ]:
character_comics()

 # Verificar
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM characters_comics")
print(f"\nTotal no banco: {cursor.fetchone()[0]} linhas")
conn.close()

[INÍCIO] Conectado ao banco de dados
[BANCO] Total de personagens encontrados: 30

[PERSONAGEM 1/30] Processando ID: 1009144
[URL] Endpoint: https://gateway.marvel.com/v1/public/characters/1009144/comics

[TENTATIVA 1/3]
[TIMEOUT] Configurado para: 10s
[PAGINAÇÃO] Offset atual: 0
[REQUISIÇÃO] Enviando para API...
[RESPOSTA] Status Code: 200
[RATE LIMIT] Requisições restantes: 3000
[COMICS] Total a processar: 53
[PAGINAÇÃO] Comics nesta página: 53
[BANCO] Inserindo 53 comics para personagem 1009144

[PERSONAGEM 2/30] Processando ID: 1009146
[URL] Endpoint: https://gateway.marvel.com/v1/public/characters/1009146/comics

[TENTATIVA 1/3]
[TIMEOUT] Configurado para: 10s
[PAGINAÇÃO] Offset atual: 0
[REQUISIÇÃO] Enviando para API...
[RESPOSTA] Status Code: 200
[RATE LIMIT] Requisições restantes: 3000
[COMICS] Total a processar: 55
[PAGINAÇÃO] Comics nesta página: 55
[BANCO] Inserindo 55 comics para personagem 1009146

[PERSONAGEM 3/30] Processando ID: 1009148
[URL] Endpoint: https://gateway.m

KeyboardInterrupt: 

# Mostrando o banco marvel_full

In [15]:
def show_table_sample():
    conn = sqlite3.connect('marvel_characters_teste.db')
    df = pd.read_sql("SELECT * FROM comics", conn)
    conn.close()

    print("\nAmostra dos dados:")
    print(f"Dimensões: {df.shape[0]} linhas x {df.shape[1]} colunas")
    return df

show_table_sample()


Amostra dos dados:
Dimensões: 63767 linhas x 8 colunas


,id,title,page_count,modified,character_collectionURI,events_collectionURI,creator_collectionURI,stories_collectionURI
0,2,Pulse (2004) #6,0,2024-09-24T09:32:11+0000,https://gateway.marvel.com/v1/public/comics/2/...,https://gateway.marvel.com/v1/public/comics/2/...,https://gateway.marvel.com/v1/public/comics/2/...,https://gateway.marvel.com/v1/public/comics/2/...
1,3,THE PULSE: THIN AIR (Trade Paperback),0,2024-07-31T10:29:21+0000,https://gateway.marvel.com/v1/public/comics/3/...,https://gateway.marvel.com/v1/public/comics/3/...,https://gateway.marvel.com/v1/public/comics/3/...,https://gateway.marvel.com/v1/public/comics/3/...
2,4,Rogue (2004) #5,0,2013-11-04T15:11:49+0000,https://gateway.marvel.com/v1/public/comics/4/...,https://gateway.marvel.com/v1/public/comics/4/...,https://gateway.marvel.com/v1/public/comics/4/...,https://gateway.marvel.com/v1/public/comics/4/...
3,5,Spectacular Spider-Man (2003) #21,0,2024-09-24T09:34:36+0000,https://gateway.marvel.com/v1/public/comics/5/...,https://gateway.marvel.com/v1/public/comics/5/...,https://gateway.marvel.com/v1/public/comics/5/...,https://gateway.marvel.com/v1/public/comics/5/...
4,6,Amazing Spider-Man (1999) #514,0,2024-09-24T09:12:59+0000,https://gateway.marvel.com/v1/public/comics/6/...,https://gateway.marvel.com/v1/public/comics/6/...,https://gateway.marvel.com/v1/public/comics/6/...,https://gateway.marvel.com/v1/public/comics/6/...
...,...,...,...,...,...,...,...,...
63762,129652,X-MEN BY CHRIS CLAREMONT: PRELUDE TO A FUTURE ...,80,2025-04-16T16:07:12+0000,https://gateway.marvel.com/v1/public/comics/12...,https://gateway.marvel.com/v1/public/comics/12...,https://gateway.marvel.com/v1/public/comics/12...,https://gateway.marvel.com/v1/public/comics/12...
63763,129884,Daredevil (2023) #20 (Variant),32,2025-02-27T09:09:43+0000,https://gateway.marvel.com/v1/public/comics/12...,https://gateway.marvel.com/v1/public/comics/12...,https://gateway.marvel.com/v1/public/comics/12...,https://gateway.marvel.com/v1/public/comics/12...
63764,129885,Daredevil (2023) #21 (Variant),32,2025-04-18T09:29:58+0000,https://gateway.marvel.com/v1/public/comics/12...,https://gateway.marvel.com/v1/public/comics/12...,https://gateway.marvel.com/v1/public/comics/12...,https://gateway.marvel.com/v1/public/comics/12...
63765,129912,Wolverines & Deadpools (2025) #1,32,2025-05-08T09:20:10+0000,https://gateway.marvel.com/v1/public/comics/12...,https://gateway.marvel.com/v1/public/comics/12...,https://gateway.marvel.com/v1/public/comics/12...,https://gateway.marvel.com/v1/public/comics/12...


In [ ]:

conn = sqlite3.connect('marvel_characters.db')
cursor = conn.cursor()

# Executar consulta com parâmetro
cursor.execute('''
    SELECT name, comics_available,series_available,	stories_available,	events_available
    FROM characters
    ORDER BY comics_available DESC
    LIMIT 10
''')

# Obter e mostrar resultados
resultados = cursor.fetchall()

In [ ]:
for name, comics_available,series_available,	stories_available,	events_available	 in resultados:
  print(f"- {name}: comics_available: {comics_available} - series_available: {series_available} - stories_available: {stories_available} - events_available: {events_available}")

- Spider-Man (Peter Parker): comics_available: 4533 - series_available: 1221 - stories_available: 6588 - events_available: 37
- X-Men: comics_available: 4113 - series_available: 1043 - stories_available: 5820 - events_available: 43
- Wolverine: comics_available: 2761 - series_available: 760 - stories_available: 3811 - events_available: 44
- Iron Man: comics_available: 2745 - series_available: 678 - stories_available: 4098 - events_available: 31
- Captain America: comics_available: 2545 - series_available: 721 - stories_available: 3723 - events_available: 30
- Avengers: comics_available: 2298 - series_available: 705 - stories_available: 3129 - events_available: 32
- Thor: comics_available: 1903 - series_available: 553 - stories_available: 2827 - events_available: 27
- Hulk: comics_available: 1777 - series_available: 547 - stories_available: 2710 - events_available: 26
- Fantastic Four: comics_available: 1564 - series_available: 456 - stories_available: 2437 - events_available: 24
- Dare